In [4]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns;


%matplotlib inline

# For Data Preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# For building the LSTM Model
from keras.models import Sequential
from keras.layers import LSTM, Dense

# For evaluating the model
from sklearn.metrics import mean_squared_error

data = pd.read_csv('Data/Powietrze_2021-2022_poprawiony_Pilsudzki.csv', sep=';')
data2 = pd.read_csv('Data/Powietrze_2021-2022_poprawiony_Rejtana.csv', index_col=False)



In [5]:
data

,Data,(pył zawieszony PM2.5 [jednostka ug/m3]),(dwutlenek azotu [jednostka ug/m3]),(tlenek węgla [jednostka ug/m3]),(benzen [jednostka ug/m3]),(tlenki azotu [jednostka ug/m3]),(tlenek azotu [jednostka ug/m3]),(pył zawieszony PM10 [jednostka ug/m3])
0,2021-01-01 01:00,45.9,8.4,0.6,1.2,11.1,1.8,53.1
1,2021-01-01 02:00,36.6,7.3,0.6,1.2,9.5,1.4,42.7
2,2021-01-01 03:00,34.6,6.3,0.5,1.1,8.8,1.6,40.5
3,2021-01-01 04:00,31.4,5.4,0.5,0.9,8.1,1.8,36.9
4,2021-01-01 05:00,25.5,3.9,0.4,0.8,7.2,2.1,30.8
...,...,...,...,...,...,...,...,...
8777,2022-01-01 19:00,8.4,20.7,0.3,3.9,37.0,10.7,12.1
8778,2022-01-01 20:00,12.0,22.5,0.3,3.9,38.0,10.1,16.2
8779,2022-01-01 21:00,15.3,14.6,0.3,3.9,28.2,8.9,19.8
8780,2022-01-01 22:00,9.5,12.9,0.2,3.8,20.4,4.9,13.3


In [9]:
data2

,Data,(tlenek azotu [jednostka ug/m3]),(ozon [jednostka ug/m3]),(dwutlenek azotu [jednostka ug/m3]),(pył zawieszony PM10 [jednostka ug/m3]),(tlenek węgla [jednostka ug/m3]),(benzen [jednostka ug/m3]),(dwutlenek siarki [jednostka ug/m3]),(tlenki azotu [jednostka ug/m3])
0,2021-01-01 01:00,0.9,20.3,6.7,,0.4,1.7,4.4,8.0
1,2021-01-01 02:00,0.7,20.2,5.8,,0.4,1.6,5.0,6.9
2,2021-01-01 03:00,0.6,21.1,5.1,,0.4,1.5,5.0,6.0
3,2021-01-01 04:00,0.6,22.5,3.8,,0.3,1.2,3.3,4.8
4,2021-01-01 05:00,0.7,23.0,3.3,,0.3,1.0,3.1,4.4
...,...,...,...,...,...,...,...,...,...
8777,2022-01-01 19:00,0.8,39.5,8.4,,0.2,0.5,2.1,9.6
8778,2022-01-01 20:00,0.7,38.9,9.6,,0.2,0.5,2.1,10.6
8779,2022-01-01 21:00,0.6,40.0,8.3,,0.2,0.6,2.1,9.3
8780,2022-01-01 22:00,0.7,47.3,6.5,,0.2,0.5,2.1,7.7


In [7]:
df = pd.merge(data, data2, on="Data")

In [8]:
df

,Data,(pył zawieszony PM2.5 [jednostka ug/m3]),(dwutlenek azotu [jednostka ug/m3])_x,(tlenek węgla [jednostka ug/m3])_x,(benzen [jednostka ug/m3])_x,(tlenki azotu [jednostka ug/m3])_x,(tlenek azotu [jednostka ug/m3])_x,(pył zawieszony PM10 [jednostka ug/m3])_x,(tlenek azotu [jednostka ug/m3])_y,(ozon [jednostka ug/m3]),(dwutlenek azotu [jednostka ug/m3])_y,(pył zawieszony PM10 [jednostka ug/m3])_y,(tlenek węgla [jednostka ug/m3])_y,(benzen [jednostka ug/m3])_y,(dwutlenek siarki [jednostka ug/m3]),(tlenki azotu [jednostka ug/m3])_y
0,2021-01-01 01:00,45.9,8.4,0.6,1.2,11.1,1.8,53.1,0.9,20.3,6.7,,0.4,1.7,4.4,8.0
1,2021-01-01 02:00,36.6,7.3,0.6,1.2,9.5,1.4,42.7,0.7,20.2,5.8,,0.4,1.6,5.0,6.9
2,2021-01-01 03:00,34.6,6.3,0.5,1.1,8.8,1.6,40.5,0.6,21.1,5.1,,0.4,1.5,5.0,6.0
3,2021-01-01 04:00,31.4,5.4,0.5,0.9,8.1,1.8,36.9,0.6,22.5,3.8,,0.3,1.2,3.3,4.8
4,2021-01-01 05:00,25.5,3.9,0.4,0.8,7.2,2.1,30.8,0.7,23.0,3.3,,0.3,1.0,3.1,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8637,2022-01-01 18:00,9.0,19.7,0.3,3.9,29.7,6.6,12.6,0.7,39.2,6.2,,0.2,0.5,2.3,7.2
8638,2022-01-01 19:00,8.4,20.7,0.3,3.9,37.0,10.7,12.1,0.8,39.5,8.4,,0.2,0.5,2.1,9.6
8639,2022-01-01 20:00,12.0,22.5,0.3,3.9,38.0,10.1,16.2,0.7,38.9,9.6,,0.2,0.5,2.1,10.6
8640,2022-01-01 21:00,15.3,14.6,0.3,3.9,28.2,8.9,19.8,0.6,40.0,8.3,,0.2,0.6,2.1,9.3
